![ASE](https://gitlab.com/ase/ase/-/raw/master/doc/static/ase256.png?inline=false)

# Introduction to the Atomic simulation environment (ASE)


ASE is a module designed for working with atoms. It uses the units of Ångstrom (Å) for length and electron volts (eV) for energy.

The central object in ASE is the [Atoms](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#module-ase.atoms)

![ASE Atoms](https://drive.google.com/uc?id=12wBZgYg7STTOYz-KNFUkbXWM9O0xsu-c)

Let's try to create an `Atoms` object the simplest way.

In [ ]:
# CO molecule with a bond length of 1.1 Å
from ase import Atoms
d = 1.1  # Å
atoms = Atoms('CO', positions=[[0, 0, 0], [0, 0, d]])

ASE contains tools to visualize the system. Use the [`view`](https://wiki.fysik.dtu.dk/ase/dev/ase/visualize/visualize.html#visualization) function. 

In [ ]:
from ase.visualize import view
view(atoms)

We can loop through the `Atoms` object to get `Atom` objects

In [ ]:
print(atoms)
print(atoms.positions)
for atom in atoms:
    print(atom)
    print(atom.index, atom.position)

As you can see, the first print statement is `Atoms`, which contains more than a single atom, while the `Atom` object only contains 1 atom. Both types of objects have variables that can be accessed directly (positions, magmoms, ...)

Let's try to setup a periodic structure.

In [ ]:
d = 2.9  # Å
L = 10  # Å
wire = Atoms('Au', positions=[[0, 0, 0]],
             cell=[d, L, L],  # unit cell lengths
             pbc=[1, 0, 0])  # periodic boundary conditions
# let's try and repeat it and visualize the repeated structure
wire10 = wire * (10, 1, 1)
view(wire10)

Let's setup a surface using one of the utility functions in [`ase.build`](https://wiki.fysik.dtu.dk/ase/dev/ase/build/build.html#module-ase.build), add an [adsorbate](https://wiki.fysik.dtu.dk/ase/dev/ase/build/surface.html#adding-adsorbates), [fix](https://wiki.fysik.dtu.dk/ase/dev/ase/constraints.html#the-fixatoms-class) the "bulk" atoms and finally do a geometrical [relaxation](https://wiki.fysik.dtu.dk/ase/dev/ase/optimize.html#module-ase.optimize)

In [ ]:
# Create the slab
from ase.build import fcc100

slab = fcc100('Cu',
              size=(3, 3, 3),
              vacuum=7)

# Uncomment the following line if you want to check that the slab looks okay
#view(slab)

In [ ]:
# Add an adsorbate
from ase.build import add_adsorbate

add_adsorbate(slab, adsorbate='Cu',
              height=3.0,
              position='ontop')

#view(slab)

In [ ]:
# Constrain the lower layers of the slab, they are the bulk
from ase.constraints import FixAtoms

con = FixAtoms(mask=[atom.tag > 1 for atom in slab])
slab.set_constraint(con)

ASE uses calculators to perform calculations. Calculators are abstract interfaces to different backends which do the actual computation. Normally, calculators work by calling an external electronic structure code or force field code. To run a calculation, we must first create a calculator and then attach it to the `Atoms` object. For the sake of the tutorial's simplicity we will use the [EMT potential](https://wiki.fysik.dtu.dk/ase/ase/calculators/emt.html#module-ase.calculators.emt) which is quite accurate for some FCC metals.

ASE provides [several optimization algorithms](https://wiki.fysik.dtu.dk/ase/ase/optimize.html#module-ase.optimize) that can run on top of `Atoms` equipped with a calculator. The `trajectory` keyword above ensures that the trajectory of intermediate geometries is written to `Cu-slab-relax.traj`.

In [ ]:
# Attach a calculator and relax the atomic positions
from ase.calculators.emt import EMT
from ase.optimize import BFGS

# The calculator
calc = EMT()
slab.calc = calc

# The optimizer
traj_file = 'Cu-slab-relax.traj'
opt = BFGS(slab, trajectory=traj_file)
opt.run(fmax=0.05)  # unit of force is eV/Å


In [ ]:
# Read in the steps of the relaxation
from ase.io import read

slab_relax = read(traj_file, index=':')

# View the adatom relax on the slab
#view(slab_relax)

# And plot the height of the adatom as a function of steps
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Get the index of the closest atom
a0 = slab_relax[0]
closest_idx = np.argmin(a0.get_distances(-1, range(len(a0) - 1)))

heights = [a.get_distance(-1, closest_idx) for a in slab_relax]
plt.plot(range(len(heights)), heights)
plt.xlabel('Steps')
plt.ylabel('Adatom height / Å')
plt.show()

In [ ]:
# add small pertubation away from the symmetric position of the adsorbate
slab[-1].position += [.1, .1, 0]
opt.run(fmax=0.05)  # Max force 0.05 eV/Å
slab_relax = read(traj_file, index=':')

view(slab_relax[-1])


We can see that the adatom has now fallen into a hollow site. Let's get the height in a different way then.

In [ ]:
heights = [a[-1].z - np.average(a[a.get_tags() == 1].positions[:, 2])
           for a in slab_relax]
plt.plot(range(len(heights)), heights)
plt.xlabel('Steps')
plt.ylabel('Adatom height / Å')
plt.show()

The current energy in eV of the `Atoms` object, given an attached calculator, can be fetched with [`get_potential_energy`](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#ase.Atoms.get_potential_energy)

In [ ]:
# get the energy out directly
print(slab.get_potential_energy())

## Molecular dynamics

Let's look at an example of an MD simulation with Langevin dynamics performed with ASE. We will still use the EMT potential, but now a faster implementation of it in [ASAP](https://wiki.fysik.dtu.dk/asap)

The Langevin dynamics will then slowly adjust the total energy of the system so the temperature approaches the desired one.

Let us melt a chunk of copper by starting the simulation without any momentum of the atoms (no kinetic energy), and with a desired temperature above the melting point. We will also save information about the atoms in a trajectory file called `moldyn3.traj`.

First we set up the structure:

In [ ]:
from ase.lattice.cubic import FaceCenteredCubic

size = 10  # Å
    
# Set up a crystal
atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          symbol='Cu',
                          size=(size, size, size),
                          pbc=True)

#view(atoms)

Then we run the MD. **Note** this calculation takes some minutes to finish, you should be able to follow the progress.

In [ ]:
from ase.io.trajectory import Trajectory
from ase.md.langevin import Langevin
from ase import units
from asap3 import EMT


# We select a temperature higher than the melting temperature of copper
T = 2000  # Kelvin
sampling_interval = 50

# Describe the interatomic interactions with the Effective Medium Theory
atoms.calc = EMT()

# We want to run MD with constant energy using the Langevin algorithm
# with a time step of 5 fs, the temperature T and the friction
# coefficient to 0.02 atomic units.
dyn = Langevin(atoms, 5 * units.fs, T * units.kB, 0.002)


def printenergy(a=atoms):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    
dyn.attach(printenergy, interval=50)

# We also want to save the positions of all atoms after every 100th time step.
traj = Trajectory('moldyn3.traj', 'w', atoms)
dyn.attach(traj.write, interval=sampling_interval)

# Now run the dynamics
printenergy()
dyn.run(5000)



### Question

* Read in the trajectory and plot the kinetic energy as a function of time.

In [ ]:
import matplotlib.pyplot as plt
from ase.io import read

%matplotlib inline



## Pourbaix diagram

A [Pourbaix diagram](https://en.wikipedia.org/wiki/Pourbaix_diagram) shows the thermodynamically stable phases of an electrochemical system in water. Let's say we would like to know what happens when MnO$_2$ dissolves in water. ASE can draw a Pourbaix diagram of the system given only a few supplied values. Many experimental solvation values are tabulated and can be fetched using [`solvated`](https://wiki.fysik.dtu.dk/ase/ase/phasediagram/phasediagram.html#ase.phasediagram.Pourbaix.decompose): 

In [ ]:
from ase.phasediagram import solvated
refs = solvated('Mn')
print(refs)

The solvated species along with their respective energies in eV are put in a list of tuples. We can add to this list our calculated/measured values.

In [ ]:
refs += [('Mn', 0.0), ('MnO2', -5.2), ('Mn2O3', -9.9), ('Mn3O4', -14.3), ('Mn(OH)2', -6.7)]

# Let's instantiate the Pourbaix object with our target system: MnO2
from ase.phasediagram import Pourbaix
pb = Pourbaix(refs, Mn=1, O=2)

Let's see what MnO$_2$ will [`decompose`](https://wiki.fysik.dtu.dk/ase/ase/phasediagram/phasediagram.html#ase.phasediagram.Pourbaix.decompose) to at a potential of 1 V and a pH of 5.0:

In [ ]:
coefs, energy = pb.decompose(U=0.5, pH=8.0)


So the reaction is: $2\text{MnO}_2 + 2\text{H}^+ + 2e^- \rightarrow \text{H}_2\text{O}  + \text{Mn}_2\text{O}_3$

To draw the full diagram do:

In [ ]:
import numpy as np

U = np.linspace(-2, 2, 200)
pH = np.linspace(-2, 12, 300)
d, names, text = pb.diagram(U, pH, plot=True)

What is the reaction at pH 10 and 1.5 V potential?

In [ ]:
pb...

# Build nanoparticles

Let us look at different ways of building nanoparticles with ASE.

## Layer specification

The first method truncates bulk structures with different surface terminations. You import the crystalstructure from `ase.cluster.cubic` and then specify the Miller indices of the exposed surfaces and number of layers. Let's try to create a small cuboctahedral particle:

In [ ]:
from ase.visualize import view
from ase.cluster.cubic import FaceCenteredCubic

surfaces = [(1, 0, 0), (1, 1, 1)]
layers = [2, 2]
lc = 3.61
atoms = FaceCenteredCubic('Cu', surfaces, layers, latticeconstant=lc)
view(atoms)

The layer specification method can be useful if you want to create a cluster where one part has been truncated by a substrate. 

In [ ]:
surfaces = [(1, 0, 0), (1, 1, 1), (1, -1, 1)]
layers = [4, 5, -1]
lc = 3.61
atoms = FaceCenteredCubic('Cu', surfaces, layers, latticeconstant=lc)
view(atoms)


More info [here.](https://wiki.fysik.dtu.dk/ase/dev/ase/cluster/cluster.html#creating-a-nanoparticle)

## Other symmetries

Creating a particle that doesn't have bulk symmetry is also possible. Let's create a small icosahedral particle:

In [ ]:
from ase.cluster import Icosahedron

atoms = Icosahedron('Cu', 3, latticeconstant=lc)
view(atoms)

It is also possible to construct decahedral structures. See [here](https://wiki.fysik.dtu.dk/ase/dev/ase/cluster/cluster.html#ase.cluster.Decahedron)

## Wulff construction

We are going to make a Pt cluster based on a Wulff construction. The Wulff construction provides the minimum energy shape for a given set of facet-dependent surface energies. In this excercise, we are going to use Wulff construction module which is avaliable in ASE for creating Pt clster. A list of surface and surface energies should be specified. The surface energies are in units of energy per area (not energy per atom). In addition, the approximate size of the nanoparticle should be given.

### Define and optimize Pt bulk structures

The EMT potential is used as a simple calculator and for quick demonstrations and tests.
   
The [`ase.build.bulk`](https://wiki.fysik.dtu.dk/ase/dev/ase/build/build.html#common-bulk-crystals) function is used to build Pt bulk structure with face center cubic (fcc) unit cell and lattice contant 3.90 Angstrom.

In [ ]:
import ase
import numpy as np
from ase.cluster import wulff_construction
from ase.visualize import view
from ase.io import write
from ase.build import bulk, surface
from ase.constraints import UnitCellFilter
from ase.optimize import BFGS
from ase.calculators.emt import EMT

atoms = bulk('Pt', 'fcc', a=3.90, cubic = True)
atoms.set_calculator(EMT())
uf = UnitCellFilter(atoms)
relax = BFGS(uf, 
             trajectory='bulk_relax.traj', 
             logfile='bulk_relax.log')
relax.run(fmax=0.01)
e_bulk = atoms.get_potential_energy()/len(atoms)
cell = atoms.get_cell()
a_bulk, b_bulk, c_bulk = np.linalg.norm(cell, axis=1)

view(atoms)

### Calculate surface energies

The [`ase.build.surface`](https://wiki.fysik.dtu.dk/ase/dev/ase/build/surface.html#create-specific-non-common-surfaces) function is used to build Pt surface from a given lattice and Miller indices
    
The surface energy can be calculated from the following expression:
    
e_surf = 0.5*(1/A)*(e_slab-n_slab*e_bulk)
    
where  e_slab is the total energy of surface slab,
     n_slab is the number of atoms in the surface slab,
     e_bulk is the bulk energy per atom,
     A is the surface area.

In [ ]:
facets = [(1, 0, 0), (1, 1, 1), (1, 1, 0), (3, 1, 1)] # A list of surfaces
esurf = []
for facet in facets:
    slab = surface(atoms, facet, 6)
    slab.center(vacuum=10, axis=2)
    slab.set_calculator(EMT())
    relax = BFGS(slab, 
                 trajectory='surface%d%d%d_relax.traj'%(facet[0],facet[1],facet[2]), 
                 logfile='surface%d%d%d_relax.log'%(facet[0],facet[1],facet[2]))
    relax.run(fmax=0.01)
    e_slab = slab.get_potential_energy()
    n_slab = len(slab)
    a = slab.get_cell()[0]
    b = slab.get_cell()[1]
    A = np.linalg.norm(np.cross(a, b))
    e_surf = 0.5*(1/A)*(e_slab-n_slab*e_bulk) # surface energy in units of energy per area 
    esurf.append(e_surf)
    print('surface energy %s : %.3f eV/Angstrom^2'%(facet, e_surf))

### Creating a cluster using Wulff construction
    
The [`ase.cluster.wulff_construction`](https://wiki.fysik.dtu.dk/ase/dev/ase/cluster/cluster.html#wulff-construction) function is used to create a cluster using the Wulff construction.
    
Arguments for function:

    symbol: The chemical symbol (or atomic number) of the desired element.
    
    surfaces: A list of surfaces. Each surface is an (h, k, l) tuple or list of integers.

    energies: A list of surface energies for the surfaces.

    size: The desired number of atoms.

    structure: The desired crystal structure. One of the strings "fcc", "bcc", or "sc".

    rounding (optional): Specifies what should be done if no Wulff construction corresponds to exactly the requested 
    number of atoms. Should be a string, either "above", "below" or "closest" (the default), meaning that the nearest 
    cluster above or below - or the closest one - is created instead.
    
You can read more about ASE modules for creating nanoparticles (clusters):
https://wiki.fysik.dtu.dk/ase/ase/cluster/cluster.html

In [ ]:
cluster = wulff_construction(symbol='Pt', 
                             surfaces=facets, 
                             energies=esurf, 
                             size=100, 
                             structure='fcc', 
                             rounding='above', 
                             latticeconstant=a_bulk)
view(cluster)

# Exercise

## Setting up a surface
Can you try to set up the (1, 1, 1) and (1, 0, 0) Pt surfaces, calculate their surface energy and include into the previous Wulff construction? How the Pt cluster change from the previous calculation and How the cluster comeout looking?

Remember to visualize your surfaces. Inspiration can be found at https://wiki.fysik.dtu.dk/ase/ase/build/surface.html

In [ ]:
import ase
import numpy as np
from ase.cluster import wulff_construction
from ase.visualize import view
from ase.io import write
from ase.build import surface







## Creating you own Au cluster: 
Can you try to make Au cluster based on Wulff construction (in the following cells) and try to answer this following queestion?:
1. Which facet give the minimum energy surface?
2. How the cluster comes out looking?

These calculations will be fast, so you can try to do the other calculations for the other elements (optinal) such as Pd, Ni and Cu (Right now, the only supported elements are: H, C, N, O, Al, Ni, Cu, Pd, Ag, Pt and Au.)

In [ ]:
import ase
import numpy as np
from ase.cluster import wulff_construction
from ase.visualize import view
from ase.io import write
from ase.build import surface
from ase.constraints import UnitCellFilter
from ase.optimize import BFGS
from ase.calculators.emt import EMT
from IPython.display import Image

""" Definie and optimize bulk structures
    Calculate the bulk energy per atom for surface energies calculation """
atoms = bulk(...)





""" Definine and optimize surface
    Calculate surface energies """
facets = [...] #A list of surfaces




""" Create a cluster using the Wulff construction and then try to visualize the cluster"""
cluster = wulff_construction(...)



